# Grover operator as a product of two level unitary gates.
$G = 2( |\psi><\psi|-I)O $ is the Grover operator with size $N\times N$. We want the following with $d=\frac{N(N-1)}{2}$

$$ U_{d} U_{d-1}...U_{2}U_{1} G = 1 $$ 

such that

$$ G = U_{1}^{\dagger}U_{2}^{\dagger}...U_{d-1}^{\dagger} U_{d}^{\dagger} $$

Where $U_{i}$ are all $N\times N$ two level unitary matrices. 

In [16]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
from numpy.linalg import matrix_power

Number of bits $n$ such that $N=2^n$.

In [17]:
n = 2

N = 2**n

## The Oracle
The oracle is a  $N \times N$ matrix with a minus sign in front of the location of the element we are looking for.

In [18]:
O = np.identity(N, dtype = complex)  

$k$ is the position of the qubit we are looking for.

In [19]:
k = 2

# The oracle matrix with a minus sign in the position of searched qubit. 
O.itemset((k,k), -1)

## The Grover operator $G = (2|\psi><\psi| -I)O$

$|\psi><\psi| = A$ is a matrix with every element $1$ in the computational basis.

In [20]:
A = np.ones((N,N))

Phase shift operation $ 2|\psi> <\psi|-I$

In [21]:
Phase = (2/N)*A - np.identity(N, dtype = complex)

The Grover operator $$ 2( |\psi><\psi|-I)O  $$

In [22]:
V = np.matmul(Phase,O)

Number of iterations
$$ M = \frac{\pi}{4} \sqrt{2^n} = \frac{\pi}{4} \sqrt{N} $$

In [23]:
Num_iteration = int(np.pi/4 * np.sqrt(N))

The operator we will unfold into two qubit gates is $$ G = (2( |\psi><\psi|-I)O)^M $$

In [24]:
G = matrix_power(V,Num_iteration)

The following function calculates two level unitary matrices for a matrix of size
greater than 3, returns a ordered pair of list
 of matrices U and the final product of all the U matrices.
 Following the algorithm given in Nielsen, M.A. and Chuang.

In [25]:
def CReductionN(Matrix):

    Two_Level_Universal_Gate = []
    
    # Extract the size of the input matrix.
    N = int(np.sqrt(Matrix.size))

    for i in range(1,N):

        if Matrix[i,0] == 0:

            U = np.identity(N, dtype = complex)

            if i == 0:
                pass
            else:
                U.itemset((0,0), np.conjugate(Matrix[0,0]))

        else:

            a, b = Matrix[0,0], Matrix[i,0]

            u00 = np.conjugate(a) / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u01 = np.conjugate(b) / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u10 = b / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u11 = - a / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
    
            U = np.identity(N, dtype = complex)
            U.itemset((0,0), u00)
            U.itemset((0,i), u01)
            U.itemset((i,0), u10)
            U.itemset((i,i), u11)
        
        Two_Level_Universal_Gate.append(U)

        # Calculating the product of all the U matrices with G.

        Matrix = np.matmul(U, Matrix)

    return Two_Level_Universal_Gate, Matrix

Function calculates two level unitary matrices for a 3x3 matrix.
    Returns a list containing the two level matrices.
    The function will be called reduced as it put zeros in the first 
    column and the first row.In what follows, the final matrix product
    will be called reduced matrix.

In [26]:
def CReduction3(Matrix):

    Two_Level_Universal_Gate = []
    
    # Size of the matrix is 3.
    N = 3
 
    for i in range(1,N):

        if Matrix[i,0] == 0:

            U = np.identity(N, dtype = complex)

            # For the first iteration it is identity.
            if i == 0:
                
                pass
            
            else:
                
                U.itemset((0,0), np.conjugate(Matrix[0,0]))

        else:

            a, b = Matrix[0,0], Matrix[i,0]

            u00 = np.conjugate(a) / np.sqrt(np.absolute(a)**2 \
                + np.absolute(b)**2)
            u01 = np.conjugate(b) / np.sqrt(np.absolute(a)**2 \
                + np.absolute(b)**2)
            u10 = b / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u11 = - a / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
    
            U = np.identity(N, dtype = complex)
            U.itemset((0,0), u00)
            U.itemset((0,i), u01)
            U.itemset((i,0), u10)
            U.itemset((i,i), u11)
        
        Two_Level_Universal_Gate.append(U)

        # Calculating product of all the U matrices with G.
        Matrix = np.matmul(U, Matrix)

    # U3 is calculated sepatarely.
    Two_Level_Universal_Gate.append(np.conjugate(Matrix).transpose())

    return Two_Level_Universal_Gate

The following function returns a sub matrix with one dimension lower than the input matrix.

In [27]:
def Extract_subMatrix(Matrix):

    N = int(np.sqrt(Matrix.size))
    subMatrix = Matrix[1:N, 1:N]

    return  subMatrix

The following function expands a given matrix to a required size.

In [28]:
def Expand_Matrix(Matrix, Required_Size):

    # Intital size of the input matrix.
    Initial_Size = int(np.sqrt(Matrix.size))

    # Creating an initial identity matrix.
    M = np.identity(Required_Size, dtype = complex)

    # Size difference between the two matrics.
    Sz = Required_Size - Initial_Size

    # The elements from the old matrix is added to M, other row and column are part of
    # identity matrix, this gurantees that the matrix is a two level unitary matrix.

    for i in range(Sz, Required_Size):
        
        for j in range(Sz, Required_Size):
            
            M.itemset((i,j), Matrix[i-Sz,j-Sz])

    return M

This loop will calculate the two level unitary matrices using the functions described above.

In [29]:
# This list will contain all the two level unitary gates.
Unitary_gates = []

Last_Reduced_Matrix = []

for i in range(N, 2, -1):

    # First iteration, start with original matrix G.
    if len(Last_Reduced_Matrix) == 0:

        M = G
        for g in CReductionN(M)[0]:
            
            Unitary_gates.append(g)

        # The reduced submatrix is put into the list to use in the next loop.
        Last_Reduced_Matrix.append(Extract_subMatrix(CReductionN(M)[1]))

    # 3x3 matrices are treated separetely with CReduction3(Matrix).
    elif i == 3:

        M = Last_Reduced_Matrix[-1]

        for g in CReduction3(M):
            
            Unitary_gates.append(g)

    else:

        # Any other dimension is reduced with CReductionN(Matrix).
        M = Last_Reduced_Matrix[-1]

        for g in CReductionN(M)[0]:
            
            Unitary_gates.append(g)   

        # The reduced submatrix is put into the list to use in the next loop.
        Last_Reduced_Matrix.append(Extract_subMatrix(CReductionN(M)[1]))

Prints all the two level unitary gates.

In [30]:
# Prints the operator.        
print('Operator being unfolded\n\n', G,'\n\n')
print('Two level unitary gates\n')
i = 1
for g in Unitary_gates:

    # If the matrix is of size NxN, print the matrix.
    if int(np.sqrt(g.size)) == N :

        print('U'+str(i)+'\n', g,'\n\n')

    else:

        # If the matrix is of lower size other than N, expand the matrix into a NxN
        # matrix and print.
        print('U'+str(i)+'\n', Expand_Matrix(g, N),'\n\n')

    i += 1    

Operator being unfolded

 [[-0.5+0.j  0.5+0.j -0.5+0.j  0.5+0.j]
 [ 0.5+0.j -0.5+0.j -0.5+0.j  0.5+0.j]
 [ 0.5+0.j  0.5+0.j  0.5+0.j  0.5+0.j]
 [ 0.5+0.j  0.5+0.j -0.5+0.j -0.5+0.j]] 


Two level unitary gates

U1
 [[-0.70710678+0.j  0.70710678-0.j  0.        +0.j  0.        +0.j]
 [ 0.70710678+0.j  0.70710678-0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  1.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  1.        +0.j]] 


U2
 [[ 0.81649658-0.j  0.        +0.j  0.57735027-0.j  0.        +0.j]
 [ 0.        +0.j  1.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.57735027+0.j  0.        +0.j -0.81649658+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  1.        +0.j]] 


U3
 [[ 0.8660254-0.j  0.       +0.j  0.       +0.j  0.5      -0.j]
 [ 0.       +0.j  1.       +0.j  0.       +0.j  0.       +0.j]
 [ 0.       +0.j  0.       +0.j  1.       +0.j  0.       +0.j]
 [ 0.5      +0.j  0.       +0.j  0.       +0